In [197]:
import numpy as np
from scipy.stats import poisson

lam = 50 #Set the mean of the Poisson distribution
p_thresh = 0.01 #Set the probability threshold

def poisson_cutoff(lam, p_thresh):
    k = int(np.floor(lam))  # start near the mean
    while True:
        pk = poisson.pmf(k, lam)
        if pk < p_thresh:
            return k - 1  # last k with P(k) >= p_thresh
        k += 1
N = poisson_cutoff(lam, p_thresh)
print(N)

63


In [198]:
k_vals = np.arange(0, N+1)
probs = poisson.pmf(k_vals, lam)
probs /= probs.sum()  # normalize

samples = np.random.choice(k_vals, size=1, p=probs)
print(samples)

[54]


In [199]:
# Send the sampled k value to the interval [0,1]
def phi(k, lam, N):
    k = np.asarray(k)
    k_star = int(np.floor(lam))
    phi_val = np.zeros_like(k, dtype=float)
    
    mask1 = k <= k_star
    phi_val[mask1] = k[mask1] / (2 * k_star)
    
    mask2 = k > k_star
    phi_val[mask2] = (k[mask2] - N) / (2 * (N - k_star)) + 1
    
    return phi_val


In [200]:
#Input Stokes vector
S_0 = np.array([1,0, 0, 1]) 

#Identity matrix
M_0 = np.array([
    [1, 0, 0, 0],
    [0 , 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
])

#Single pm fiber matrix
M_12 = np.array([
    [1, 0, 0, 0],
    [0.627773 , -0.261825, 0.45438, 0.480219],
    [-0.708459, -0.293815, -0.307685, -0.02616],
    [1.12193, 0.989887, -0.191137, -0.015808]
])

#Bit flip matrix
M_1 = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, -1]
])



In [201]:
#Linear interpolation between the matrices
def M_t(t, M_0, M_12, M_1):
    t = np.asarray(t)  # allow scalar or array input

    # allocate output array
    if t.ndim == 0:  # scalar
        if 0 <= t <= 0.5:
            return M_0 + 2*(M_12 - M_0)*t
        elif 0.5 < t <= 1:
            return M_1 + 2*(M_12 - M_1)*(1 - t)
        else:
            raise ValueError("t must be in [0,1]")
    else:  # array of t
        M_t = np.empty((t.size,) + M_0.shape, dtype=M_0.dtype)
        mask1 = t <= 0.5
        mask2 = t > 0.5
        M_t[mask1] = M_0 + 2*(M_12 - M_0)*t[mask1,None,None]
        M_t[mask2] = M_1 + 2*(M_12 - M_1)*(1 - t[mask2,None,None])
        return M_t

In [212]:
#Randomly pick a value between 0 and N, report the probability of
#obtaining that value and the associated Mueller matrix, as well as
#the output Stokes vector
k = samples = np.random.choice(k_vals, size=1, p=probs)
phi_val = phi(k, lam, N)[0]
p = poisson.pmf(k, lam)
M = M_t(phi_val, M_0, M_12, M_1)
S_1 = M@S_0
print(f"With probability {p}, choose {k}, \n giving Mueller matrix \n {M} \n and Stokes vector \n {S_1}.")

With probability [0.05632501], choose [49], 
 giving Mueller matrix 
 [[ 1.          0.          0.          0.        ]
 [ 0.61521754 -0.2365885   0.4452924   0.47061462]
 [-0.69428982 -0.2879387  -0.2815313  -0.0256368 ]
 [ 1.0994914   0.97008926 -0.18731426  0.00450816]] 
 and Stokes vector 
 [ 1.          1.08583216 -0.71992662  1.10399956].
